https://www.lazard.com/research-insights/2023-levelized-cost-of-energyplus/

https://www.energy.gov/eere/solar/solar-photovoltaic-system-cost-benchmarks

https://www.integratesun.com/post/how-much-does-it-cost-to-install-solar-panels-in-2025-1

battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price *  (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [ ]:
%pip install pyomo

In [2]:
import pyomo.environ as pyo

In [5]:
model = pyo.ConcreteModel()

model.solar_capacities = pyo.Set(initialize=[3, 5, 6, 8, 10, 12, 15])
model.select_solar_capacity = pyo.Var(model.solar_capacities, within=pyo.Binary)
model.solar_capacity = pyo.Var()
model.peak_grid_consumption = pyo.Var(within=pyo.NonNegativeReals)
model.off_peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)
model.battery_capacity = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 4))
model.peak_load = pyo.Param(initialize=1)
model.off_peak_load = pyo.Param(initialize=5)



model.peak_grid_price = pyo.Param(initialize=0.5)
model.off_peak_grid_price = pyo.Param(initialize=0.5)
model.solar_cost_per_kw = pyo.Param(initialize=0.1)
model.battery_cost_per_kw = pyo.Param(initialize=0.1)

model.minimizeCost = pyo.Objective(
    expr=(model.peak_grid_price * model.peak_grid_consumption)
    + (model.off_peak_grid_price * model.off_peak_grid_usage)
    + (model.solar_cost_per_kw * model.solar_capacity)
    + (model.battery_cost_per_kw * model.battery_capacity),
    sense=pyo.minimize,
)

def pick_one(m):
    return 1 == sum(m.select_solar_capacity[i] for i in m.solar_capacities)
model.pick_one = pyo.Constraint(rule=pick_one)



model.off_peak_constraint = pyo.Constraint(
    expr=model.off_peak_load <= model.off_peak_grid_usage + model.battery_capacity
)

model.peak_constraint = pyo.Constraint(
    expr=model.peak_load <= model.peak_grid_consumption + model.solar_capacity
)

model.battery_charging_constraint = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)

def set_solar_capacity(m):
    return m.solar_capacity == sum(i*m.select_solar_capacity[i] for i in m.solar_capacities)
model.set_solar_capacity = pyo.Constraint(rule=set_solar_capacity)


optimizer = pyo.SolverFactory("gurobi")
optimizer.solve(model)
print(
    f"solar capacity: {model.solar_capacity.value} kW, battery capacity: {model.battery_capacity.value} kWh, off peak grid usage: {model.off_peak_grid_usage.value} kW, peak grid consumption: {model.peak_grid_consumption.value} kW"
)
print(model.display())

solar capacity: 5.0 kW, battery capacity: 4.0 kWh, off peak grid usage: 1.0 kW, peak grid consumption: 0.0 kW
Model unknown

  Variables:
    select_solar_capacity : Size=7, Index=solar_capacities
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          3 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary
         12 :     0 :   0.0 :     1 : False : False : Binary
         15 :     0 :   0.0 :     1 : False : False : Binary
    solar_capacity : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   5.0 :  None : False : False :  Reals
    peak_grid_consumption : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : Fal

In [8]:
model.solar_capacity.display()

solar_capacity : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  five :  None : three : False :  True : {three, four, five}
